# Reducción del dataset

En este notebook vamos a realizar una reducción del dataset original, donde explicamos las razones de esta decisión y el proceso llevado a cabo.

En la fase 1 del proyecto (Identificación y estudio teórico de datos industriales) analizamos la estructura del dataset *Tennessee Eastman Process (TEP)*. Este esta compuesto por simulaciones industriales que representan tanto el comportamiento normal como los 20 distintos tipos de fallos.

En ese análisis detallamos las variables que forman parte del proceso, comprobamos que no había valores nulos, y confirmamos que el volumen de datos del dataset era enorme (cientos de miles de registros), especialmente para los archivos de fallos.

Este tamaño es ideal para realizar una investigación avanzada, pero para desarrollar nuestro proyecto, nos supone grandes limitaciones. Por ejemplo, usar el dataste completo afecta en los tiempos de carga de los archivos, retrasa el entrenamiento y dificulta la experimentación del usuario en los paneles.

Por estas razones, antes de avanzar con los siguientes pasos del proyecto, entre ellos, realizar un análisis exploratorio más profundo, la ingeniería de características y creación de modelos, vamos a **reducir el dataset original**.

#### Estrategia para la reducción

Para conservar el proceso industrial pero sin usar todas las simulaciones del dataset, vamos a usar un **muestreo a nivel de simulación**, estratificado por tipo de fallo en los dataset que tienen fallos, y muestreo aleatorio de simulaciones en los datasets sin fallos. De esta forma mantenemos el proceso completo, solo reducimos el número de simulaciones.

Mantenemos todas la columnas originales del conjunto de datos original y los segmentos temporales (conservamos las simulaciones completas). Pero tambien creamos una nueva variable binaria, *fault_present*, para indicar si la simulación pertenece a un escenario de fallo o no. 
Para los datasets Faulty vamos a seleccionar 10 simulaciones por tipo de fallo y por otro lado, para los datasets de Fault-Free, vamos a seleccionar 20 simulaciones. Esto lo hacemos sin mezclar las condiciones, si el archivo solo tiene fallos, se muestrean solo simulaciones con fallo, y lo mismo en caso contrario.

De esta forma garantizamos que los datos reducidos mantienen la dimensionalidad original (las 52 variables del proceso) y reflejamos ambas condiciones, las normales y con los fallos simulados.

### Faulty_Training y Faulty_Testing

Comenzamos reduciendo los archivos que tienen las simulaciones con fallos.

Primero cargamos los archivos .RData y convertimos las columnas de tipo float64 a float32, de esta forma reducimos memoria. Después creamos una etiqueta para indicar si hay fallo o no (fault_present). El siguiente paso es identificar la columna que define las simulaciones (simulationRun) y aplicamos el muestreo adecuado dependiendo de si el archivo tiene o no tiene fallos. Por último creamos y guardamos los nuevos archivos reducidos.


In [1]:
import pyreadr
import pandas as pd
import numpy as np
import os

def load_and_reduce(file_path, n_sims_per_fault=10, n_sims_faultfree=20):
    try:
        result = pyreadr.read_r(file_path)
        df_name = list(result.keys())[0]
        df = result[df_name]

        # Pasar a float32
        for col in df.select_dtypes(include="float64").columns:
            df[col] = df[col].astype("float32")

        # Crear etiqueta
        df['fault_present'] = (df['faultNumber'] > 0).astype(int)

        # Detectar columna de simulación
        sim_col = "simulationRun"

        # Comprobar si el dataset tiene fallos
        fault_values = df["faultNumber"].unique()
        has_faults = any(fault_values > 0)

        # CASO 1 — DATASET FAULTY
        if has_faults:
            df_faulty = df[df["faultNumber"] > 0]

            # muestrear n simulaciones por tipo de fallo
            selected_faulty_sims = (
                df_faulty.groupby("faultNumber")[sim_col]
                .apply(lambda sims: sims.drop_duplicates().sample(
                    n=min(n_sims_per_fault, len(sims)), random_state=42))
                .explode()
                .reset_index(drop=True)
            )

            df_reduced = df[df[sim_col].isin(selected_faulty_sims)]

            return df_reduced, sim_col

        # CASO 2 — DATASET FAULT-FREE
        else:
            selected_sims = (
                df[sim_col].drop_duplicates().sample(
                    n=min(n_sims_faultfree, df[sim_col].nunique()),
                    random_state=42
                )
            )

            df_reduced = df[df[sim_col].isin(selected_sims)]
            return df_reduced, sim_col

    except Exception as e:
        print("ERROR load_and_reduce:", e)
        return pd.DataFrame(), None


# Ejemplo de uso completo
DATA_DIR = os.getcwd()
DEST_DIR = os.path.join(DATA_DIR, "DatasetReducido")
os.makedirs(DEST_DIR, exist_ok=True)

file_paths = {
    "Faulty Training": os.path.join(DATA_DIR, "TEP_Faulty_Training.RData"),
    "Faulty Testing": os.path.join(DATA_DIR, "TEP_Faulty_Testing.RData"),
}

for name, path in file_paths.items():
    df_reduced, sim_col = load_and_reduce(path)
    if not df_reduced.empty:
        out_file = os.path.join(DEST_DIR, f"{name.replace(' ', '_')}_reduced.csv")
        df_reduced.to_csv(out_file, index=False)
        print(f"\n--- {name} reducido ---")
        print("Archivo guardado en:", out_file)
        print("Simulaciones:", df_reduced[sim_col].nunique())
        print("Muestras:", len(df_reduced))
        print("Columnas:", df_reduced.columns.tolist())

print("\nTodos los datasets reducidos guardados en la carpeta DatasetReducido.")



--- Faulty Training reducido ---
Archivo guardado en: C:\Users\elbab\Documents\ProyectoAnalitica\proyecto_analitica\DatasetReducido\Faulty_Training_reduced.csv
Simulaciones: 10
Muestras: 100000
Columnas: ['faultNumber', 'simulationRun', 'sample', 'xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11', 'fault_present']

--- Faulty Testing reducido ---
Archivo guardado en: C:\Users\elbab\Documents\ProyectoAnalitica\proyecto_analitica\DatasetReducido\Faulty_Testing_redu

### FaultFree_Training y FaultFree_Testing

Para estos otros dos archivos Fault-Free, que no contienen fallos usamos la misma función de reducción que utilizamos para los datasets con fallos. Como en este caso no hay distintos tipos de fallo, solo aplicamos una selección aleatoria para limitar el número de simulaciones. Cargamos los datasets, convertimos las columnas de tipo float64 a float32. De esta forma creamos un dataset reducido conservando las columnas originales y la estructura de las simulaciones.

In [6]:
import pyreadr
import pandas as pd
import numpy as np
import os

# def load_and_reduce(file_path, n_sims_per_fault=10, n_sims_faultfree=20):
#     try:
#         result = pyreadr.read_r(file_path)
#         df_name = list(result.keys())[0]
#         df = result[df_name]

#         # Pasar a float32
#         for col in df.select_dtypes(include="float64").columns:
#             df[col] = df[col].astype("float32")

#         # Crear etiqueta
#         df['fault_present'] = (df['faultNumber'] > 0).astype(int)

#         # Detectar columna de simulación
#         sim_col = "simulationRun"

#         # Comprobar si el dataset tiene fallos
    #     fault_values = df["faultNumber"].unique()
    #     has_faults = any(fault_values > 0)

    #     # CASO 1 — DATASET FAULTY
    #     if has_faults:
    #         df_faulty = df[df["faultNumber"] > 0]

    #         # muestrear n simulaciones por tipo de fallo
    #         selected_faulty_sims = (
    #             df_faulty.groupby("faultNumber")[sim_col]
    #             .apply(lambda sims: sims.drop_duplicates().sample(
    #                 n=min(n_sims_per_fault, len(sims)), random_state=42))
    #             .explode()
    #             .reset_index(drop=True)
    #         )

    #         df_reduced = df[df[sim_col].isin(selected_faulty_sims)]

    #         return df_reduced, sim_col

    #     # CASO 2 — DATASET FAULT-FREE
    #     else:
    #         selected_sims = (
    #             df[sim_col].drop_duplicates().sample(
    #                 n=min(n_sims_faultfree, df[sim_col].nunique()),
    #                 random_state=42
    #             )
    #         )

    #         df_reduced = df[df[sim_col].isin(selected_sims)]
    #         return df_reduced, sim_col

    # except Exception as e:
    #     print("ERROR load_and_reduce:", e)
    #     return pd.DataFrame(), None

# Ejemplo de uso completo
DATA_DIR = os.getcwd()
DEST_DIR = os.path.join(DATA_DIR, "DatasetReducido")
os.makedirs(DEST_DIR, exist_ok=True)

file_paths = {
    "FaultFree Training": os.path.join(DATA_DIR, "TEP_FaultFree_Training.RData"),
    "FaultFree Testing": os.path.join(DATA_DIR, "TEP_FaultFree_Testing.RData"),
}

for name, path in file_paths.items():
    df_reduced, sim_col = load_and_reduce(path)
    if not df_reduced.empty:
        out_file = os.path.join(DEST_DIR, f"{name.replace(' ', '_')}_reduced.csv")
        df_reduced.to_csv(out_file, index=False)
        print(f"\n--- {name} reducido ---")
        print("Archivo guardado en:", out_file)
        print("Simulaciones:", df_reduced[sim_col].nunique())
        print("Muestras:", len(df_reduced))
        print("Columnas:", df_reduced.columns.tolist())


print("\nTodos los datasets reducidos guardados en la carpeta DatasetReducido.")


--- FaultFree Training reducido ---
Archivo guardado en: C:\Users\elbab\Documents\ProyectoAnalitica\proyecto_analitica\DatasetReducido\FaultFree_Training_reduced.csv
Simulaciones: 20
Muestras: 10000
Columnas: ['faultNumber', 'simulationRun', 'sample', 'xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11', 'fault_present']

--- FaultFree Testing reducido ---
Archivo guardado en: C:\Users\elbab\Documents\ProyectoAnalitica\proyecto_analitica\DatasetReducido\FaultFree_T

Después de realizar este proceso de reducción de datos, vamos a comprobar los resultados cargando los datasets de training reducidos.

In [7]:
import pandas as pd

df_faulty = pd.read_csv("DatasetReducido/Faulty_Training_reduced.csv")
df_faultfree = pd.read_csv("DatasetReducido/FaultFree_Training_reduced.csv")

print("FAULTY:")
print(df_faulty["fault_present"].value_counts())
print("Simulaciones únicas:", df_faulty["simulationRun"].nunique())

print("\nFAULTFREE:")
print(df_faultfree["fault_present"].value_counts())
print("Simulaciones únicas:", df_faultfree["simulationRun"].nunique())

FAULTY:
fault_present
1    100000
Name: count, dtype: int64
Simulaciones únicas: 10

FAULTFREE:
fault_present
0    10000
Name: count, dtype: int64
Simulaciones únicas: 20


Al analizar los resultados podemos ver como las condiciones de los dataset son las correctas segun la reducción que hemos realizado. 

El dataset de Faulty Training, en la columna fault_present solo tiene el valor 1, esto indica que en este nuevo archivo solo hay simulaciones que tienen fallos, y cuenta con 10 simulaciones únicas.
El dataset FaultFree Training solo tiene el valor 0 en la columna fault_present, y asi se verifica que el dataset solo tiene simulaciones (normales) sin fallos. En este caso son 20 simulaciones únicas.

Una vez hemos realizado este proceso para los cuatro archivos del dataste TEP, obtenemos un conjunto de datos reducido, pero representativo y eficiente. Este dataset nuevo será el que usaremos para el resto del proyecto.